In [9]:
import sys
import subprocess
import os
import yt.wrapper as yt

import torch
import torch.nn as nn

sys.path.append('../proto')
from realty.offer import common_pb2, RealtySchema_pb2
from realty.offer.ranking import factors_pb2, embedding_pb2
from google.protobuf.json_format import MessageToDict

from time import time
from tqdm import tqdm


yt.config["proxy"]["url"] = "hahn"

from io import BytesIO

    from google.protobuf.internal.encoder import _EncodeVarint
from google.protobuf.internal.decoder import _DecodeVarint

def parseFromDelimitedString(obj, buf, offset=0):
    """
    Stanford CoreNLP uses the Java "writeDelimitedTo" function, which
    writes the size (and offset) of the buffer before writing the object.
    This function handles parsing this message starting from offset 0.
    @returns how many bytes of @buf were consumed.
    """
    size, pos = _DecodeVarint(buf, offset)
    obj.ParseFromString(buf[(offset+pos):(offset+pos+size)])
    return pos+size

def writeToDelimitedString(obj, stream=None):
    """
    Stanford CoreNLP uses the Java "writeDelimitedTo" function, which
    writes the size (and offset) of the buffer before writing the object.
    This function handles parsing this message starting from offset 0.
    @returns how many bytes of @buf were consumed.
    """
    if stream is None:
        stream = BytesIO()

    _EncodeVarint(stream.write, obj.ByteSize(), True)
    stream.write(obj.SerializeToString())
    return stream

In [125]:
table_path="//home/verticals/realty/production/export/offers/snapshot"
sampling_rate = 1

res_proto = [] 
stream = BytesIO()

output_file = '../proto.request.binary'

for index, row in enumerate(yt.read_table(table_path,
                                                   format=yt.YsonFormat(encoding=None),
                                                   table_reader={"sampling_seed": 42,
                                                                 "sampling_rate": sampling_rate})):
    o = row[b'offer']
    
    request = RealtySchema_pb2.OfferMessage()
    request.ParseFromString(o)
    res_proto.append(request)
#     writeToDelimitedString(request, stream)
    
#     with open(output_file, 'w+b') as file:
#         file.write(stream.getvalue())
    
    if index == 10:
        break
        
buf = stream.getvalue()
stream.close()

In [236]:
import sys

from flask import Response
from google.protobuf.internal.decoder import _DecodeVarint
from google.protobuf.json_format import MessageToDict

from time import time

sys.path.append('./proto')
from realty.offer import RealtySchema_pb2


#############################################################################
# responses for json and protobuf


def ParseFromDelimitedString(obj, buf, offset=0):
    size, pos = _DecodeVarint(buf, offset)
    obj.ParseFromString(buf[(offset+pos):(offset+pos+size)])
    return pos+size



def parse_protobuf(buf):
    dicts = []
    offset = 0
    while offset < len(buf):
        emb = EmbeddingTransport()
        offset += ParseFromDelimitedString(emb, buf[offset:])
        
        dicts.append(emb)
    return dicts

In [237]:
with open('../response_97', 'rb') as f:
    buf = f.read()

In [238]:
embs = parse_protobuf(buf)

In [239]:
embs[0]

offer_id: "8281984907028075264"
timestamp {
  seconds: 1594370324
}
dssm_embedding {
  model_name: "DSSM"
  values: -0.023386210203170776
  values: 0.00617777556180954
  values: -0.012072216719388962
  values: -0.1171167716383934
  values: 0.06185802072286606
  values: -0.07538369297981262
  values: -0.07816453278064728
  values: -0.06341046094894409
  values: 0.10431039333343506
  values: 0.04161439836025238
  values: -0.03261672705411911
  values: -0.06923767924308777
  values: 0.032505977898836136
  values: 0.061657968908548355
  values: -0.027796758338809013
  values: 0.07537376880645752
  values: -0.08591672778129578
  values: -0.07285898923873901
  values: 0.04550042003393173
  values: -0.0287624541670084
  values: 0.04341369494795799
  values: -0.046087246388196945
  values: 0.0476284921169281
  values: -0.03452952951192856
  values: -0.05883825942873955
  values: -0.1026555597782135
  values: -0.0070157842710614204
  values: -0.06439430266618729
  values: -0.1264011561870575
  

In [132]:
RealtySchema_pb2.RawImageMessage

RealtySchema_pb2.RawImageMessage

In [ ]:
message['raw_image'][0]['cv_meta']['similarity_vector']['feature']

In [150]:
offer = RealtySchema_pb2.OfferMessage()
ParseFromDelimitedString(offer, buf[0:])
message_dict = MessageToDict(offer,
                                 preserving_proto_field_name=True)
# protobuf_to_dict(offer)

### Результат

In [211]:
embe = Embedding()
embe.model_name = 'pe'
embe.values.extend([1.,2.,3.])

embTr = EmbeddingTransport()
embTr.offer_id = '12345'
embTr.dssm_embedding.CopyFrom(embe)

embTr

offer_id: "12345"
dssm_embedding {
  model_name: "pe"
  values: 1.0
  values: 2.0
  values: 3.0
}

In [218]:
time()

1594368411.137183

In [224]:
embTr.timestamp.From

In [222]:
embTr.timestamp.ToDatetime()

datetime.datetime(2020, 7, 10, 8, 6, 51)

### BigB

In [42]:
from ads.bsyeti.libs.log_protos import universal_update_pb2
from yabs.proto import user_profile_pb2

entry = universal_update_pb2.TProfileUpdate()
entry.yandex_uid = 12345
action = entry.commands.add()
items = action.add.items.add()

setattr(items, "keyword_id", 968)
setattr(items, "string_value", b"kek")

entry

commands {
  add {
    items {
      keyword_id: 968
      string_value: "kek"
    }
  }
}
yandex_uid: 12345

In [172]:
def write_request_proto_to_file(request, output_file='proto.request.example.binary'):
    global result
    result = request.SerializeToString()
    with open(output_file, 'w+b') as file:
        file.write(result)

32819

In [156]:


entry


commands {
  add {
    items {
      keyword_id: 968
      string_value: "kek"
    }
  }
}
yandex_uid: 12345

In [217]:
items

keyword_id: 968
string_value: "kek"

In [173]:
s = writeToDelimitedString(request)

In [178]:
serialized = request.SerializeToString()

In [180]:
serialized

b'\x08\x10\x10\xa4\xd7\xb7\xe2\x85\x9c\xbb\xae\x07\x18\xa4\xd7\xb7\xe2\x85\x9c\xbb\xae\x07 \x010\x028\xac\x95\x8f\xa2\xa7.@\xf8\x90\xc0\xf4\xa9.H\xc4\xed\xec\xfd\x03R\x88\x08\x08\x02\x10C\x1a\x07\x88\xff\xa4\x08\x90\xdc\n2\x0b\x08\x01\x10\xbc\xab5\x1d\x00`\x07E2\x0b\x08\x01\x10\xad\xab5\x1d\x00\x90\x0cE2\x0b\x08\x01\x10\x91\xac5\x1d\x00\x90;E:\n\xd0\xa2\xd0\xbe\xd0\xbc\xd1\x81\xd0\xbaB\x1d\xd0\x9a\xd0\xbe\xd0\xbb\xd1\x85\xd0\xbe\xd0\xb7\xd0\xbd\xd0\xb0\xd1\x8f \xd1\x83\xd0\xbb\xd0\xb8\xd1\x86\xd0\xb0J\x039/1Z\x0c\x08\x01\x15\xaf\xeaaB\x1d,\x02\xaaBb\x0c\x08\x01\x15\x00\x00z\xc4\x1d\x00\x00z\xc4\x82\x01<\xd0\xa0\xd0\xbe\xd1\x81\xd1\x81\xd0\xb8\xd1\x8f, \xd0\xa2\xd0\xbe\xd0\xbc\xd1\x81\xd0\xba, \xd0\x9a\xd0\xbe\xd0\xbb\xd1\x85\xd0\xbe\xd0\xb7\xd0\xbd\xd0\xb0\xd1\x8f \xd1\x83\xd0\xbb\xd0\xb8\xd1\x86\xd0\xb0, 9/1\x8a\x01P\xd0\xa0\xd0\xbe\xd1\x81\xd1\x81\xd0\xb8\xd1\x8f, \xd0\xa2\xd0\xbe\xd0\xbc\xd1\x81\xd0\xba\xd0\xb0\xd1\x8f \xd0\xbe\xd0\xb1\xd0\xbb\xd0\xb0\xd1\x81\xd1\x82\xd1\x8c, \xd0\x

In [159]:
def buffer_to_messages(msg, buf):
    messages = []
    offset = 0
    while offset < len(buf):
        offer = msg
        offset += parseFromDelimitedString(offer, buf[offset:])
        messages.append(offer)
        
    return messages

In [160]:
result = buffer_to_messages(RealtySchema_pb2.OfferMessage(), buf)

23864 3
27846 3
6392 2
64159 3
15766 2
26755 3
65775 3
37740 3
10114 2
2693 2
32819 3


In [168]:
result[0].ranking_factors.cat_factors[2]

factor_name: PRICE_TREND
factor_value: 3

In [154]:
for string, proto in zip(res, result):
    o = RealtySchema_pb2.OfferMessage()
    o.ParseFromString(string)
    
    print(o == proto)


True
True
True
True
True
True
True
True
True
True
True


In [153]:
string

b'\x08\x10\x10\xa4\xd7\xb7\xe2\x85\x9c\xbb\xae\x07\x18\xa4\xd7\xb7\xe2\x85\x9c\xbb\xae\x07 \x010\x028\xac\x95\x8f\xa2\xa7.@\xf8\x90\xc0\xf4\xa9.H\xc4\xed\xec\xfd\x03R\x88\x08\x08\x02\x10C\x1a\x07\x88\xff\xa4\x08\x90\xdc\n2\x0b\x08\x01\x10\xbc\xab5\x1d\x00`\x07E2\x0b\x08\x01\x10\xad\xab5\x1d\x00\x90\x0cE2\x0b\x08\x01\x10\x91\xac5\x1d\x00\x90;E:\n\xd0\xa2\xd0\xbe\xd0\xbc\xd1\x81\xd0\xbaB\x1d\xd0\x9a\xd0\xbe\xd0\xbb\xd1\x85\xd0\xbe\xd0\xb7\xd0\xbd\xd0\xb0\xd1\x8f \xd1\x83\xd0\xbb\xd0\xb8\xd1\x86\xd0\xb0J\x039/1Z\x0c\x08\x01\x15\xaf\xeaaB\x1d,\x02\xaaBb\x0c\x08\x01\x15\x00\x00z\xc4\x1d\x00\x00z\xc4\x82\x01<\xd0\xa0\xd0\xbe\xd1\x81\xd1\x81\xd0\xb8\xd1\x8f, \xd0\xa2\xd0\xbe\xd0\xbc\xd1\x81\xd0\xba, \xd0\x9a\xd0\xbe\xd0\xbb\xd1\x85\xd0\xbe\xd0\xb7\xd0\xbd\xd0\xb0\xd1\x8f \xd1\x83\xd0\xbb\xd0\xb8\xd1\x86\xd0\xb0, 9/1\x8a\x01P\xd0\xa0\xd0\xbe\xd1\x81\xd1\x81\xd0\xb8\xd1\x8f, \xd0\xa2\xd0\xbe\xd0\xbc\xd1\x81\xd0\xba\xd0\xb0\xd1\x8f \xd0\xbe\xd0\xb1\xd0\xbb\xd0\xb0\xd1\x81\xd1\x82\xd1\x8c, \xd0\x

In [58]:
import io

In [59]:
stream = writeToDelimitedString(request)
buf = stream.getvalue()
stream.close()

# doc_pb_ = RealtySchema_pb2.OfferMessage()
# parseFromDelimitedString(doc_pb_, buf)
# assert request == doc_pb_